### Hotel Review Classification

In [191]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
import os, sys
import matplotlib.pyplot as plt
import time
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
import re
import spacy
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,MaxPooling1D,Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM,Bidirectional
from keras.layers.convolutional import Conv1D
nltk.download('stopwords')
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords


# from spacy.cli import download
# print(download('en')
#
#nlp = spacy.load('en')

#import nltk


[nltk_data] Downloading package stopwords to /Users/mosal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## EDA

In [139]:
# function for reading data from the url


In [218]:
hotel_train = pd.read_csv('Data/sentiment_dataset_train.csv')
hotel_dev = pd.read_csv('Data/sentiment_dataset_dev.csv')

In [248]:
hotel_train.head()
hotel_train = hotel_train[hotel_train['rating'] !='Tables not made up prior to guest seating. 2.']

In [249]:
hotel_train.shape

(35004, 3)

In [250]:
hotel_train = hotel_train.dropna()

In [256]:
X_train = hotel_train['review']
y_train = hotel_train['rating']

X_dev = hotel_dev['review']
y_dev = hotel_dev['rating']

In [257]:
X_train

0        Arrived about 10pm and check in was painless. ...
1        I checked in at 4pm even tough room was not re...
2        I chose this hotel, as it was in a good locati...
3        Great location, super close to shops & a 10min...
4        I was in the Sir Adam Hotel to visit a friend....
                               ...                        
35000    Paris is always welcome city, but this time th...
35001    Beautiful place very clean and welcoming irini...
35002    The hotel is ok considering the price we paid....
35003    First your stuck if you miss last tram at midn...
35004    The staff was very nice. The room was fine - n...
Name: review, Length: 35004, dtype: object

In [262]:
y_train.dtype

dtype('int64')

In [261]:
y_train = y_train.astype(int)

In [265]:
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train).toarray() 

In [227]:
y_train

0        4
1        2
2        2
3        4
4        3
        ..
35000    5
35001    3
35002    3
35003    3
35004    3
Name: rating, Length: 35005, dtype: object

In [231]:
#y_train = hotel_train['rating']


In [ ]:
# fitting the model
lgr = LogisticRegression(max_iter=500, penalty='l2', random_state=None,  solver='lbfgs').fit(X_train_vec, y_train)
# printing the result
print('Train accuracy = ',lgr.score(X_train_vec, y_train))

In [232]:
### BEGIN STARTER CODE
class DLTextClassifier():
    def __init__(self, 
                 embedding_dimension = 200,
                 max_features = 20000, 
                 maxlen = 80):
        """
        Instantiate the DLTextClassifer.              
        Parameters:
        ------------------        
        embedding_dimension : (int)
            size of your word embedding vector
        max_features: (int)
            max number of words to keep in the vocabulary
        maxlen : (int)
            sequence length
        """
        # We'll be using an embedding layer and pass a vector of
        # size embedding dimension instead of one-hot-encoding. 
        self.embedding_dimension = embedding_dimension
        self.max_features = max_features    
        # Sequence length
        self.maxlen = maxlen
        # Create the tokenizer. We'll be using Keras Tokenizer here.         
        self.tokenizer = Tokenizer(num_words=self.max_features, 
                             filters='! #$% ()*+,-./:; = ?@[\\]^_`{|}~\t\n>"<') 
        # Store word_index
        self.word_index = self.tokenizer.word_index
                
    def prepare_data(self, corpus, mode = 'train'):
        """
        Create a mapping from the unique words in the training corpus to integers, 
        the output then passed to pad_sequences function to give all the sequence
        the same length. 

        Parameters:
        ------------------ 
        corpus :  list of texts
        mode : train or test

        Returns:
        ------------------ 
        padded sequences of the corpus
        print the length of the encoded corpus
        """
        if mode == 'train': 
            # fit the tokenizer on the documents
            self.tokenizer.fit_on_texts(corpus)
        
        # integer encoded documents
        encoded_corpus = self.tokenizer.texts_to_sequences(corpus)        
        print('len of encoded docs: ', len(encoded_corpus))
        return self.pad_sequences(encoded_corpus)

    def pad_sequences(self, data):
        """
        This function will call pad_sequences function to add 0 to the sequence
        in the data that has shorter length than the maximum length and print the
        shape of the padded data.

        Parameters:
        ------------------ 
        data: List of lists and each element is a sequence
        Returns :
        ------------------ 
        padded data : Array of the padded data    
        """
       
        print('Pad sequences (samples x time)')
        padded_data = sequence.pad_sequences(data, maxlen=self.maxlen)
        print('Padded data shape:', padded_data.shape)
        return padded_data    
          
    def build_network(self, layer_size = 256, dropout_amount=0.5):
        """
        Given layer_size and dropout_amount, build an LSTM network
        using Keras and tensorflow and print summary of the model. 

        Parameters
        -----------
        layer_size : int
          The number of units to be passed in the LSTM layer
        dropout_amount : float
          the dropout amount to be passed in the Dropout layer. 

        Return
        -----------
        None
          print the summary of the model 
        """      
    
        model = Sequential()
        model.add(Embedding(self.max_features, self.embedding_dimension,
                            input_length=self.maxlen))
        model.add(LSTM(layer_size, dropout=dropout_amount,
                       recurrent_dropout=dropout_amount,return_sequences=True))
        model.add(LSTM(layer_size,return_sequences=True))
        model.add(LSTM(layer_size))
        model.add(Dense(1, activation='sigmoid')) 
        model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
        self.model=model
        print(model.summary())
               
    def fit(self, 
              X_train, y_train,
              batch_size =32, 
              epochs = 10, 
              save_path='/content/gdrive/My Drive/'):
        """        
        Given the parameters train a deep learning model and save and return it.  
        
        Parameters
        -------------
        X_train : (list) 
          the X values of the train split 
        y_train : (list) 
          the y values of the train split 
        batch_size : (int) 
          the batch_size for the training
        epochs : (int) 
          the number of epochs for training 
        save_path : (str) the path to save the model
        
        Return
        -------------
          the trained model
        """      
        # Directory where the checkpoints will be saved
        checkpoint_dir = save_path
        # Name of the checkpoint files
        checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
        checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_prefix,
            save_weights_only=True)
        X_train_padded = self.prepare_data(X_train, mode='train')
        self.model.fit(X_train_padded, y_train,epochs=epochs, 
         batch_size=batch_size, 
         callbacks=[checkpoint_callback])
        self.model.save(save_path)

    def evaluate(self, X_test, y_test):
        """
        Given a model and X_test, y_test, evaluates the model and prints 
        the accuracy
        
        Parameters
        -----------
        X_test -- Array of text data
        y_test -- labels for every text
        
        Return:    
        -----------    
        None                
        """        
        X_test_padded = self.prepare_data(X_test, mode='test')        
        score, acc = self.model.evaluate(X_test_padded, y_test)
        print('Accuracy: ', acc)


    def predict(self, texts):
        """
        Return predicted labels for a list of texts.        

        Parameters
        -----------
        
        texts : (list) 
          a list of text comments

        Return:
        --------        
          a list of prediction scores        
        """        
        padded_sequences = self.prepare_data(texts, mode = 'test')
        return self.model.predict(padded_sequences)
### END STARTER CODE    

In [178]:
### YOUR ANSWER HERE
def preprocess(text, 
               min_token_len = 2, 
               irrelevant_pos = ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']): 
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text 
    and return a preprocessed string. 
    
    Parameters
    -------------
    text : (str) 
        the text to be preprocessed
    min_token_len : (int) 
        min_token_length required
    irrelevant_pos : (list) 
        a list of irrelevant pos tags
    
    Returns
    -------------
    (str) the preprocessed text
    """
    #cleaning the data
    text=text.replace(r'\n','')
    stop_words = list(set(stopwords.words('english')))+list(string.punctuation)  # all stop_words and punctuation
    stop_words.extend(['``','’', '`','br','"',"”", "''", "'s"])
    tokens = word_tokenize(text)
    processed_text = [] 
    for token in tokens:
        if token not in stop_words:
          processed_text.append(str(token))
    final_txt = " ".join(processed_text)
    return final_txt
### END STARTER CO

In [244]:
X_train = X_train['review']

In [192]:
X_train['preproc_text'] = X_train.review.apply(preprocess)

<ipython-input-192-3411e0ce6ae4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['preproc_text'] = X_train.review.apply(preprocess)


In [194]:
X_train = X_train['preproc_text']

In [195]:
X_train

0        Arrived 10pm check painless The downside hotel...
1        I checked 4pm even tough room ready .. staff b...
2        I chose hotel good location room bath spa avai...
3        Great location super close shops 10min walk ma...
4        I Sir Adam Hotel visit friend We enjoyed time ...
                               ...                        
35000    Paris always welcome city time I stay Hotel Co...
35001    Beautiful place clean welcoming irini wonderfu...
35002    The hotel ok considering price paid Breakfast ...
35003    First stuck miss last tram midnight cheapest w...
35004    The staff nice The room fine spectacular small...
Name: preproc_text, Length: 35005, dtype: object

In [264]:
# # Task 2 : Planned/No planned
# model_p =DLTextClassifier()
# model_p.build_network()
# t = time.time()
# model_p.fit(X_train,y_train,batch_size =128,epochs = 1)
# elapsed_time = time.time() - t
# print(elapsed_time)